In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import re
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("ggplot")

In [2]:
url ="https://www.trustpilot.com/review/expondo.com"
# 3. download html with a get request. Use the function request.get() and store the output in response
response = requests.get(url)
# 200 status code means OK! response.status_code
print(response.status_code)

200


In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
# 4.2. check that the html code looks like it should
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="/manifest.json" rel="manifest"/>
  <meta content="Trustpilot" name="application-name"/>
  <meta content="#1c1c1c" name="theme-color"/>
  <link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link color="#00b67a" href="https://cdn.trustpilot.net/brand-assets/4.3.0/favicons/safari-pinned-tab.svg" rel="mask-icon"/>
  <meta content="Trustpilot" name="apple-mobile-web-app-title"/

In [4]:
# With groupping of lists:

from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime as dt

# Initialize lists
review_id = []
review_title = []
review_date = []
review_rating = []
review_content = []
review_location = []
page_number = []

# Set Trustpilot page numbers to scrape here
from_page = 1
to_page = 12

for i in range(from_page, to_page + 1):
    #url = f"https://www.trustpilot.com/review/expondo.com?page={i}"
    url=f"https://www.trustpilot.com/review/expondo.com?page={i}&sort=recency"
    response = requests.get(url)
    response.status_code

    soup = BeautifulSoup(response.text, 'html.parser')

    for review in soup.find_all('section', class_='styles_reviewsContainer__3_GQw'):
        
        # review_id
        rev_id = [re.search(r'/reviews/([a-f\d]+)', elem['href']).group(1) for elem in soup.select("div.styles_reviewContent__0Q2Tg a") if elem.parent]
        review_id.extend(rev_id)
        
        # title:
        title=[elem.get_text().replace("\n","").replace("\t","")for elem in soup.select("div a h2")]
        review_title.extend(title)
                
        # date:
        
        date = [
            datetime.strptime(elem.get_text().replace("\n", "").replace("\t", "").replace("Date of experience: ",""), "%B %d, %Y").strftime("%d/%m/%Y")
            for elem in soup.select("article section div p.typography_body-m__xgxZ_")
        ]
        review_date.extend(date)
        
        # rating
        
        rating = [
            int(elem['data-service-review-rating'])
            for elem in soup.select('.styles_reviewHeader__iU9Px[data-service-review-rating]')
        ]
        review_rating.extend(rating)
        
        # content
        
       # content = [
       #     elem.get_text() 
       #     for elem in soup.select('section div p.typography_body-l__KUYFJ.typography_appearance-default__AAY17[ data-service-review-text-typography="true"]')
       # ]

        for div_element in soup.find_all('div', class_='styles_reviewContent__0Q2Tg'):
            p_element = div_element.find('p', class_='typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn', attrs={'data-service-review-text-typography': 'true'})

            if p_element:
                review_text = p_element.get_text(strip=True)
                review_content.append(review_text)
            else:
                #print("")
                review_content.append("")
        
        # location
        
        location = [
            elem.get_text(strip=True)
            for elem in soup.select("article div.typography_appearance-subtle__8_H2l span")
            if elem.find_parent("div", class_="styles_consumerExtraDetails__fxS4S")
        ]
        review_location.extend(location)
        
        page_number.append(i)

In [5]:
print(len(review_id))
print(len(review_title))
print(len(review_date))
print(len(review_rating))
print(len(review_content))
print(len(review_location))

240
240
240
240
240
240


In [6]:
review_df = pd.DataFrame({
    'review_id': review_id,
    'review_title': review_title,
    'review_date': review_date,
    "review_rating":review_rating,
    "review_content":review_content,
    "review_location":review_location
})

In [7]:
review_df['review_date'] = pd.to_datetime(review_df['review_date'], format='%d/%m/%Y')

In [8]:
review_df=review_df.sort_values(by="review_date",ascending=False).reset_index(drop=True)

In [9]:
review_df.head()

,review_id,review_title,review_date,review_rating,review_content,review_location
0,65aed6b600af4883acaf8caf,"TIG (WIG) welder, model S-WIGMA 200PDC",2024-01-22,5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",AT
1,65a68b82126d92f1f9a4d330,Awesome experience,2024-01-15,5,Awesome experience. We had to use their custom...,BG
2,65a3c4680208eaa47b381c4d,The product arrived on time all the…,2024-01-12,5,The product arrived on time all the assembly d...,IE
3,659fcc8b2e14391e7d9d5597,Good and fast delivery,2023-12-27,5,Good and fast delivery of this amazing standin...,CZ
4,659e780423cb48162da0eabb,Bad invoice cross-border purchase - no contact...,2023-12-26,1,Bad invoice - DPH on invoice - shipping to slo...,SK


In [10]:
review_df.to_csv("expondo_reviews_since_2023.csv",index=False)

## Sentiment analysis

### Step 1. VADER Seniment Scoring

Step 1. VADER Seniment Scoring
We will use NLTK's SentimentIntensityAnalyzer to get the neg/neu/pos scores of the text.

This uses a "bag of words" approach:
Stop words are removed
each word is scored and combined to a total score.

In [11]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

In [12]:
sia.polarity_scores('I am so happy!')

{'neg': 0.0, 'neu': 0.318, 'pos': 0.682, 'compound': 0.6468}

In [13]:
sia.polarity_scores('This is the worst thing ever.')

{'neg': 0.451, 'neu': 0.549, 'pos': 0.0, 'compound': -0.6249}

In [14]:
# Run the polarity score on the entire dataset
res = {}
for i, row in tqdm(review_df.iterrows(), total=len(review_df)):
    text = row['review_content']
    myid = row['review_id']
    res[myid] = sia.polarity_scores(text)

  0%|          | 0/240 [00:00<?, ?it/s]

In [15]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'id'})
display(vaders.shape)
display(vaders.head())

(230, 5)

,id,neg,neu,pos,compound
0,65aed6b600af4883acaf8caf,0.02,0.879,0.102,0.9091
1,65a68b82126d92f1f9a4d330,0.00,0.552,0.448,0.9299
2,65a3c4680208eaa47b381c4d,0.00,0.571,0.429,0.9147
3,659fcc8b2e14391e7d9d5597,0.00,0.586,0.414,0.8908
4,659e780423cb48162da0eabb,0.39,0.530,0.080,-0.7920


In [16]:
display(review_df.shape)
vaders = vaders.merge(review_df, how='left',left_on="id",right_on="review_id")
display(vaders.head())
vaders.shape

(240, 6)

,id,neg,neu,pos,compound,review_id,review_title,review_date,review_rating,review_content,review_location
0,65aed6b600af4883acaf8caf,0.02,0.879,0.102,0.9091,65aed6b600af4883acaf8caf,"TIG (WIG) welder, model S-WIGMA 200PDC",2024-01-22,5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",AT
1,65a68b82126d92f1f9a4d330,0.00,0.552,0.448,0.9299,65a68b82126d92f1f9a4d330,Awesome experience,2024-01-15,5,Awesome experience. We had to use their custom...,BG
2,65a3c4680208eaa47b381c4d,0.00,0.571,0.429,0.9147,65a3c4680208eaa47b381c4d,The product arrived on time all the…,2024-01-12,5,The product arrived on time all the assembly d...,IE
3,659fcc8b2e14391e7d9d5597,0.00,0.586,0.414,0.8908,659fcc8b2e14391e7d9d5597,Good and fast delivery,2023-12-27,5,Good and fast delivery of this amazing standin...,CZ
4,659e780423cb48162da0eabb,0.39,0.530,0.080,-0.7920,659e780423cb48162da0eabb,Bad invoice cross-border purchase - no contact...,2023-12-26,1,Bad invoice - DPH on invoice - shipping to slo...,SK


(240, 11)

## Step 2. Roberta Pretrained Model

Use a model trained of a large corpus of data.
Transformer model accounts for the words but also the context related to other words.

In [17]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [18]:
from transformers import pipeline

In [19]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [20]:
review_df.head()

,review_id,review_title,review_date,review_rating,review_content,review_location
0,65aed6b600af4883acaf8caf,"TIG (WIG) welder, model S-WIGMA 200PDC",2024-01-22,5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",AT
1,65a68b82126d92f1f9a4d330,Awesome experience,2024-01-15,5,Awesome experience. We had to use their custom...,BG
2,65a3c4680208eaa47b381c4d,The product arrived on time all the…,2024-01-12,5,The product arrived on time all the assembly d...,IE
3,659fcc8b2e14391e7d9d5597,Good and fast delivery,2023-12-27,5,Good and fast delivery of this amazing standin...,CZ
4,659e780423cb48162da0eabb,Bad invoice cross-border purchase - no contact...,2023-12-26,1,Bad invoice - DPH on invoice - shipping to slo...,SK


In [21]:
def polarity_scores_roberta(review_content):
    encoded_text = tokenizer(review_content, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [22]:
res = {}
for i, row in tqdm(review_df.iterrows(), total=len(review_df)):
    try:
        text = row['review_content']
        myid = row["review_id"]
        vader_result = sia.polarity_scores(text)
        vader_result_rename = {}
        for key, value in vader_result.items():
            vader_result_rename[f"vader_{key}"] = value
        roberta_result = polarity_scores_roberta(text)
        both = {**vader_result_rename, **roberta_result}
        res[myid] = both
    except RuntimeError:
        print(f'Broke for id {myid}')

  0%|          | 0/240 [00:00<?, ?it/s]

In [23]:
results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index': 'id'})
results_df.head()
results_df = results_df.merge(review_df,left_on="id",right_on="review_id")

In [24]:
display(results_df.head())
display(results_df.shape)

,id,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,review_id,review_title,review_date,review_rating,review_content,review_location
0,65aed6b600af4883acaf8caf,0.02,0.879,0.102,0.9091,0.042506,0.260303,0.697191,65aed6b600af4883acaf8caf,"TIG (WIG) welder, model S-WIGMA 200PDC",2024-01-22,5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",AT
1,65a68b82126d92f1f9a4d330,0.00,0.552,0.448,0.9299,0.006276,0.030150,0.963574,65a68b82126d92f1f9a4d330,Awesome experience,2024-01-15,5,Awesome experience. We had to use their custom...,BG
2,65a3c4680208eaa47b381c4d,0.00,0.571,0.429,0.9147,0.002630,0.015183,0.982187,65a3c4680208eaa47b381c4d,The product arrived on time all the…,2024-01-12,5,The product arrived on time all the assembly d...,IE
3,659fcc8b2e14391e7d9d5597,0.00,0.586,0.414,0.8908,0.001461,0.015258,0.983281,659fcc8b2e14391e7d9d5597,Good and fast delivery,2023-12-27,5,Good and fast delivery of this amazing standin...,CZ
4,659e780423cb48162da0eabb,0.39,0.530,0.080,-0.7920,0.833544,0.153574,0.012882,659e780423cb48162da0eabb,Bad invoice cross-border purchase - no contact...,2023-12-26,1,Bad invoice - DPH on invoice - shipping to slo...,SK


(240, 14)


# Function to determine sentiment based on Roberta scores
def determine_sentiment_roberta(row):
    if row['roberta_pos'] > row['roberta_neu'] and row['roberta_pos'] > row['roberta_neg']:
        return 'positive'
    elif row['roberta_neu'] > row['roberta_pos'] and row['roberta_neu'] > row['roberta_neg']:
        return 'neutral'
    else:
        return 'negative'

# Add a new column 'roberta_sentiment' to the DataFrame
df['roberta_sentiment'] = df.apply(determine_sentiment, axis=1)

# Display the updated DataFrame
print(df)


In [25]:
def analyze_sentiment_vader(text):
    analyzer = SentimentIntensityAnalyzer()
    compound_score = analyzer.polarity_scores(text)['compound']
    if compound_score >= 0.05:
        return 'Positive'
    elif compound_score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

In [26]:
def analyze_sentiment_roberta(text):
    classifier = pipeline('sentiment-analysis', model="cardiffnlp/twitter-roberta-base-sentiment")
    result = classifier(text)
    sentiment_label=result[0]['label']
    sentiment_map = {'LABEL_0': 'Negative', 'LABEL_1': 'Neutral', 'LABEL_2': 'Positive'}
    mapped_sentiment = sentiment_map.get(sentiment_label, 'Unknown')
    return mapped_sentiment

In [27]:
results_df['sentiment_roberta'] = results_df['review_content'].apply(analyze_sentiment_roberta)

In [28]:
results_df['sentiment_vader'] = results_df['review_content'].apply(analyze_sentiment_vader)

In [29]:
results_df.head()

,id,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,review_id,review_title,review_date,review_rating,review_content,review_location,sentiment_roberta,sentiment_vader
0,65aed6b600af4883acaf8caf,0.02,0.879,0.102,0.9091,0.042506,0.260303,0.697191,65aed6b600af4883acaf8caf,"TIG (WIG) welder, model S-WIGMA 200PDC",2024-01-22,5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",AT,Positive,Positive
1,65a68b82126d92f1f9a4d330,0.00,0.552,0.448,0.9299,0.006276,0.030150,0.963574,65a68b82126d92f1f9a4d330,Awesome experience,2024-01-15,5,Awesome experience. We had to use their custom...,BG,Positive,Positive
2,65a3c4680208eaa47b381c4d,0.00,0.571,0.429,0.9147,0.002630,0.015183,0.982187,65a3c4680208eaa47b381c4d,The product arrived on time all the…,2024-01-12,5,The product arrived on time all the assembly d...,IE,Positive,Positive
3,659fcc8b2e14391e7d9d5597,0.00,0.586,0.414,0.8908,0.001461,0.015258,0.983281,659fcc8b2e14391e7d9d5597,Good and fast delivery,2023-12-27,5,Good and fast delivery of this amazing standin...,CZ,Positive,Positive
4,659e780423cb48162da0eabb,0.39,0.530,0.080,-0.7920,0.833544,0.153574,0.012882,659e780423cb48162da0eabb,Bad invoice cross-border purchase - no contact...,2023-12-26,1,Bad invoice - DPH on invoice - shipping to slo...,SK,Negative,Negative


## Step 3. TextBlob

In [30]:
from textblob import TextBlob

In [31]:
def analyze_sentiment(text):
    analysis = TextBlob(text)
    # Classify polarity as positive, negative, or neutral
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity < 0:
        return 'Negative'
    else:
        return 'Neutral'

In [32]:
# Apply sentiment analysis to the 'Review Content' column
results_df['sentiment_textblob'] = results_df['review_content'].apply(analyze_sentiment)

In [33]:
# Display the resulting DataFrame
display(results_df)

,id,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,review_id,review_title,review_date,review_rating,review_content,review_location,sentiment_roberta,sentiment_vader,sentiment_textblob
0,65aed6b600af4883acaf8caf,0.020,0.879,0.102,0.9091,0.042506,0.260303,0.697191,65aed6b600af4883acaf8caf,"TIG (WIG) welder, model S-WIGMA 200PDC",2024-01-22,5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",AT,Positive,Positive,Positive
1,65a68b82126d92f1f9a4d330,0.000,0.552,0.448,0.9299,0.006276,0.030150,0.963574,65a68b82126d92f1f9a4d330,Awesome experience,2024-01-15,5,Awesome experience. We had to use their custom...,BG,Positive,Positive,Positive
2,65a3c4680208eaa47b381c4d,0.000,0.571,0.429,0.9147,0.002630,0.015183,0.982187,65a3c4680208eaa47b381c4d,The product arrived on time all the…,2024-01-12,5,The product arrived on time all the assembly d...,IE,Positive,Positive,Positive
3,659fcc8b2e14391e7d9d5597,0.000,0.586,0.414,0.8908,0.001461,0.015258,0.983281,659fcc8b2e14391e7d9d5597,Good and fast delivery,2023-12-27,5,Good and fast delivery of this amazing standin...,CZ,Positive,Positive,Positive
4,659e780423cb48162da0eabb,0.390,0.530,0.080,-0.7920,0.833544,0.153574,0.012882,659e780423cb48162da0eabb,Bad invoice cross-border purchase - no contact...,2023-12-26,1,Bad invoice - DPH on invoice - shipping to slo...,SK,Negative,Negative,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,639d7d5bd075435bd8ded454,0.000,0.622,0.378,0.8442,0.002801,0.018716,0.978483,639d7d5bd075435bd8ded454,I have bought Christmas gift,2022-12-14,5,I have bought Christmas gift. It was delivered...,CZ,Positive,Positive,Positive
236,63a8193c2338b6d417a6995c,0.000,1.000,0.000,0.0000,0.008240,0.350710,0.641050,63a8193c2338b6d417a6995c,Quick reply to inquiries,2022-12-12,5,Quick reply to inquiries. Fast delivery.,HU,Positive,Neutral,Positive
237,63a20c632338b6d417a21d48,0.042,0.746,0.212,0.7731,0.015204,0.097120,0.887675,63a20c632338b6d417a21d48,customer service,2022-12-07,5,this is hard time for all of us. still when i ...,GB,Positive,Positive,Positive
238,639df97beb0ad848990e73cc,0.043,0.806,0.151,0.7096,0.845883,0.140066,0.014050,639df97beb0ad848990e73cc,Shady..,2022-12-04,1,This company promise free delivery in Portugal...,PT,Negative,Positive,Positive


In [34]:
bins = [0, 2, 3, 5]
labels = ['Negative', 'Neutral', 'Positive']

# Create a new column 'sentiment' based on the bins
results_df['rating_sentiment'] = pd.cut(results_df['review_rating'], bins=bins, labels=labels, include_lowest=True)


In [35]:
results_df.head()

,id,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,review_id,review_title,review_date,review_rating,review_content,review_location,sentiment_roberta,sentiment_vader,sentiment_textblob,rating_sentiment
0,65aed6b600af4883acaf8caf,0.02,0.879,0.102,0.9091,0.042506,0.260303,0.697191,65aed6b600af4883acaf8caf,"TIG (WIG) welder, model S-WIGMA 200PDC",2024-01-22,5,"I ordered a TIG (WIG) / MMA welder, model S-WI...",AT,Positive,Positive,Positive,Positive
1,65a68b82126d92f1f9a4d330,0.00,0.552,0.448,0.9299,0.006276,0.030150,0.963574,65a68b82126d92f1f9a4d330,Awesome experience,2024-01-15,5,Awesome experience. We had to use their custom...,BG,Positive,Positive,Positive,Positive
2,65a3c4680208eaa47b381c4d,0.00,0.571,0.429,0.9147,0.002630,0.015183,0.982187,65a3c4680208eaa47b381c4d,The product arrived on time all the…,2024-01-12,5,The product arrived on time all the assembly d...,IE,Positive,Positive,Positive,Positive
3,659fcc8b2e14391e7d9d5597,0.00,0.586,0.414,0.8908,0.001461,0.015258,0.983281,659fcc8b2e14391e7d9d5597,Good and fast delivery,2023-12-27,5,Good and fast delivery of this amazing standin...,CZ,Positive,Positive,Positive,Positive
4,659e780423cb48162da0eabb,0.39,0.530,0.080,-0.7920,0.833544,0.153574,0.012882,659e780423cb48162da0eabb,Bad invoice cross-border purchase - no contact...,2023-12-26,1,Bad invoice - DPH on invoice - shipping to slo...,SK,Negative,Negative,Negative,Negative


In [36]:
results_df.to_csv("results_for sentiment.csv",index=False,sep=";")

In [ ]:
results_df.to_csv("results_for sentiment2.csv",index=False,sep=";")

In [ ]:
from sklearn.metrics import classification_report

print("TextBlob:")
print(classification_report(results_df['rating_sentiment'], results_df['sentiment_textblob']))

print("\nVADER:")
print(classification_report(results_df['rating_sentiment'], results_df['sentiment_vader']))

print("\nRoBERTa:")
print(classification_report(results_df['rating_sentiment'], results_df['sentiment_roberta']))


In [54]:
# With groupping of lists:

from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime as dt

# Initialize lists
review_id_mowdirect = []
review_title_mowdirect = []
review_date_mowdirect = []
review_rating_mowdirect = []
review_content_mowdirect = []
review_location_mowdirect= []
page_number = []

# Set Trustpilot page numbers to scrape here
from_page = 16
to_page = 30

for i in range(from_page, to_page + 1):
    #url = f"https://www.trustpilot.com/review/expondo.com?page={i}"
    #url=f"https://www.trustpilot.com/review/www.amazon.com?page={i}&stars=3"
    url=f"https://www.trustpilot.com/review/www.mowdirect.co.uk?page={i}&stars=3"
    response = requests.get(url)
    response.status_code

    soup = BeautifulSoup(response.text, 'html.parser')

    for review in soup.find_all('section', class_='styles_reviewsContainer__3_GQw'):
        
        # review_id
        rev_id_mowdirect= [re.search(r'/reviews/([a-f\d]+)', elem['href']).group(1) for elem in soup.select("div.styles_reviewContent__0Q2Tg a") if elem.parent]
        review_id_mowdirect.extend(rev_id_mowdirect)
        
        # title:
        title_mowdirect=[elem.get_text().replace("\n","").replace("\t","")for elem in soup.select("div a h2")]
        review_title_mowdirect.extend(title_mowdirect)
        
                
        # date:
        
        date_mowdirect = [
            datetime.strptime(elem.get_text().replace("\n", "").replace("\t", "").replace("Date of experience: ",""), "%B %d, %Y").strftime("%d/%m/%Y")
            for elem in soup.select("article section div p.typography_body-m__xgxZ_")
        ]
        review_date_mowdirect.extend(date_mowdirect)
        
        # rating
        
        rating_mowdirect = [
            int(elem['data-service-review-rating'])
            for elem in soup.select('.styles_reviewHeader__iU9Px[data-service-review-rating]')
        ]
        review_rating_mowdirect.extend(rating_mowdirect)
        
        # content
        
       # content = [
       #     elem.get_text() 
       #     for elem in soup.select('section div p.typography_body-l__KUYFJ.typography_appearance-default__AAY17[ data-service-review-text-typography="true"]')
       # ]

        for div_element in soup.find_all('div', class_='styles_reviewContent__0Q2Tg'):
            p_element = div_element.find('p', class_='typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn', attrs={'data-service-review-text-typography': 'true'})

            if p_element:
                review_text = p_element.get_text(strip=True)
                review_content_mowdirect.append(review_text)
            else:
                #print("")
                review_content_mowdirect.append("")
        
        # location
        
        location_mowdirect = [
            elem.get_text(strip=True)
            for elem in soup.select("article div.typography_appearance-subtle__8_H2l span")
            if elem.find_parent("div", class_="styles_consumerExtraDetails__fxS4S")
        ]
        review_location_mowdirect.extend(location_mowdirect)
        
        
        page_number.append(i)

In [39]:
review_df_amazon = pd.DataFrame({
    'review_id': review_id_amazon,
    'review_title': review_title_amazon,
    'review_date': review_date_amazon,
    "review_rating":review_rating_amazon,
    "review_content":review_content_amazon,
    "review_location":review_location_amazon
})

In [40]:
review_df_amazon

,review_id,review_title,review_date,review_rating,review_content,review_location
0,65b115680916f543b1a389a3,Bad experience with marketplace seller,05/01/2024,2,"Unfortunately I had a bad experience which, on...",US
1,65afdb177d8134ffaae6549a,I always buy products from Amazon,01/01/2024,1,"I always buy products from Amazon, BUT if ther...",US
2,65b02d374f07df2dd9dad8b5,"No help as usual, this time with the new “One ...",23/01/2024,1,I buy products but getting any help is incredi...,US
3,65b246b77650506acff6d3b2,Touching rock bottom in 2024,24/01/2024,1,Amazon used to provide great service until 202...,GB
4,65b252c6dbd119ab2c125594,Well,25/01/2024,4,"Well, y'all lot know what it is. It is the one...",DE
...,...,...,...,...,...,...
395,5eea0d5f7dd75308d0567904,9832102621,17/06/2020,3,Thank you for,IN
396,5ee64d227dd7530708834ae0,Product not delivered,14/06/2020,3,Amazon customer care number 8247721430 produc...,IN
397,5edbc15025e5d209b8fea308,I buy a lot of stuff from Amazon and…,06/06/2020,3,I buy a lot of stuff from Amazon and they are ...,US
398,5edb39e425e5d209b8fe3e25,Listen to the customer,06/06/2020,3,Dispence with companies who make misleading co...,GB


In [42]:
review_df_amazon.to_csv("./data/neutral_first_20.csv",index=False)

In [44]:
review_df_amazon2 = pd.DataFrame({
    'review_id': review_id_amazon,
    'review_title': review_title_amazon,
    'review_date': review_date_amazon,
    "review_rating":review_rating_amazon,
    "review_content":review_content_amazon,
    "review_location":review_location_amazon
})

In [45]:
review_df_amazon2

,review_id,review_title,review_date,review_rating,review_content,review_location
0,5ed3e15925e5d209b8f930c6,I love the system of delivery and order…,31/05/2020,3,I love the system of delivery and order but th...,US
1,5ed1135325e5d209b8f7c4d9,The prime fee is not worth it any…,29/05/2020,3,"The prime fee is not worth it any longerAlso, ...",US
2,5ecd776d25e5d209b8f54806,Displeased,26/05/2020,3,Overall I do not have a problem with Amazon. A...,US
3,5ec9c85125e5d20a889dc5d4,What's going on with Amazon deliveries lately?,24/05/2020,3,I ordered two of the exact same sweatshirts fr...,US
4,5ec8b6d425e5d20a889d223f,Lately Amazon have got absurdly high…,23/05/2020,3,Lately Amazon have got absurdly high prices on...,AU
...,...,...,...,...,...,...
415,4e7091f100006400020e2a07,Prices can be high,14/09/2011,3,Try Book Depository as prices are frequentky c...,GB
416,4e4a8c7100006400020cfd0a,Amazon confusion,16/08/2011,3,Dont get me wrong I like Amazon. Sometimes the...,GB
417,4e3a1ad400006400020c84e4,I enjoy shopping with them,04/08/2011,3,"I think the order was professionally handled, ...",US
418,4d2a8cc2000064000206f77e,Amazon doesnt ship to India!,10/01/2011,3,I really love the product range at Amazon.comH...,IN


In [46]:
review_df_amazon2.to_csv("./data/neutral_second_20.csv",index=False)

In [51]:
review_df_mowdirect  = pd.DataFrame({
    'review_id': review_id_mowdirect ,
    'review_title': review_title_mowdirect ,
    'review_date': review_date_mowdirect ,
    "review_rating":review_rating_mowdirect ,
    "review_content":review_content_mowdirect ,
    "review_location":review_location_mowdirect 
})

In [52]:
review_df_mowdirect.head()

,review_id,review_title,review_date,review_rating,review_content,review_location
0,65a685af5ffe4c8b43e58099,Wow great service.,15/01/2024,5,"Wow, what a great service. This is my third pu...",GB
1,65ae3abee0553e01f9c3d3a1,Started well then hopeless,12/01/2024,1,I saw a recon Racing mower advertised that met...,GB
2,65a66c6bc190cee2d2b2cda5,absolute rubbish back up when things go…,04/01/2024,1,"absolute rubbish back up when things go wrong,...",GB
3,656b5cbcc7c5b22c61cea87c,Goods received after a week,22/11/2023,4,"Goods received after a week. That’s ok, but no...",GB
4,65a6c631cc3b0333dd876352,Instructions are a bit basic,06/01/2024,4,Instructions are a bit basic! so you need basi...,GB


In [53]:
review_df_mowdirect.to_csv("./data/mowdirect1.csv",index=False)

In [55]:
review_df_mowdirect2  = pd.DataFrame({
    'review_id': review_id_mowdirect ,
    'review_title': review_title_mowdirect ,
    'review_date': review_date_mowdirect ,
    "review_rating":review_rating_mowdirect ,
    "review_content":review_content_mowdirect ,
    "review_location":review_location_mowdirect 
})

In [56]:
review_df_mowdirect2.to_csv("./data/mowdirect2.csv",index=False)